In [ ]:
pip install langchain_community

# LLM model


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)

In [ ]:
#use Mistral llm model from hugging face if you want to.

'''
from google.colab import userdata
from langchain_community.llms import HuggingFaceHub
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
llm=HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.1",huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)
,,,

# Embedding model


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
pip install langchain_google_genai

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
vector = embeddings.embed_query("hello, world!")


In [ ]:
len(vector) #for configurign the pinecone this value is important.

768

In [ ]:
pip install langchain_pinecone

In [ ]:
pip install langchain_community

In [ ]:
pip install langchain_text_splitters

# Data

In [ ]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
loader = TextLoader("/content/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Initializing Pinecone

In [ ]:
from google.colab import userdata
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')

In [ ]:
vectorstore = PineconeVectorStore(index_name="ragsystem", embedding=embeddings,pinecone_api_key=PINECONE_API_KEY)

In [ ]:
 vectorstore.add_documents(docs)

['147d7592-8eec-4f66-8af5-c6b60f42b1d4',
 'e5f398f4-d417-4521-b206-c451accdc7e7',
 '5a501a5d-9a1a-408c-8220-c1296b6abc0b',
 '5dc5a088-b1c3-446c-a789-b6c2de3b26b3',
 '135f80c3-34a4-4996-91d4-809f6879eea5',
 '850f91ec-0592-4b16-aa3e-e3423caf899c',
 '05587d9c-6f83-4481-a64d-5b2159c2d650',
 '3b4be748-354e-4a1b-beab-921f429b217a',
 '124cb4f0-9a7b-4ab5-b3ac-268f7dd900f9',
 'f8118317-3518-41d4-9aa7-48253f307e7a',
 '752fc4e3-743b-417d-ab17-ed8e518de151',
 '1b8cb5a8-782d-43c6-bc81-551a29e4a657',
 'f92548f9-fff0-41d4-82db-0c190e2c40eb',
 'd3cdbad2-f864-45dd-9fef-9a2791b2d92b',
 '243b96ff-6d7c-4dca-8bb6-de3ba351d826',
 '596f8a93-958d-463d-b3ba-8a0e123bacc3',
 'f1767a72-84bc-4750-a946-d18a6596ae65',
 '9aacfb1a-d2b2-43b8-b814-73c2c49a9dd6',
 '1ee9bb20-02c1-4d70-a47e-5c98d648a773',
 'fe53a2ef-86a5-4bb0-afad-ea9f18a7b7c2',
 'd5886b84-759e-4030-ba0e-9f9a820537e5',
 'b12ab6c9-1507-4e7a-8cdb-0825c74cf444',
 'a878dc4d-3a01-4d9b-b70c-90f91c054ebd',
 '2b71d117-06a3-42f2-8ded-fc70d89c9606',
 '1eda97fe-64ae-

In [ ]:
 query = "Who is Ketanji Brown Jackson?"
print(vectorstore.similarity_search(query)[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


# RAG chain

In [ ]:
pip install langchain

In [ ]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context
and keep your answer consis:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser

chain.invoke("who is Ketanji Brown Jackson?")

'Ketanji Brown Jackson is a Circuit Court of Appeals Judge who was nominated by President Biden to serve on the United States Supreme Court.'

In [ ]:
chain.invoke("what is the meaning of life?")

'The provided context does not contain information about the meaning of life, so I cannot answer this question from the provided context.'